In [30]:
import requests
#from lxml import etree, html
import re
from bs4 import BeautifulSoup

In [85]:
url = "https://www.nytimes.com/content/help/rights/privacy/policy/privacy-policy.html"
page = requests.get(url)
content = BeautifulSoup(page.content,'html.parser')

In [86]:
pars = [p.prettify() for p in content.find_all('p')]
pars = [p.replace('\n','') for p in pars] #Remove new lines

In [87]:
pars

['<p> <nyt_displaydate> </nyt_displaydate></p>',
 '<p> <em>  Last Updated on May 18, 2017 </em></p>',
 '<p> This Privacy Policy discloses the privacy practices for The New York Times newspaper and NYTimes.com (including international.nytimes.com, the online edition of The International New York Times), <a href="http://homedelivery.nytimes.com/HDS/HDSHome.do?mode=HDSHome">  The New York Times Home Delivery </a> website, <a href="http://nytneediestcases.com">  The New York Times Neediest Cases Fund </a> , <a href="http://www.nytstore.com/" title="Welcome to The New York Times Store - Rare and Newsworthy Items">  The New York Times Store </a> , <a href="http://www.nytimes.com/times-journeys/">  Times Journeys </a> and <a href="http://learning.blogs.nytimes.com/" title="The Learning Network - The Learning Network Blog - NYTimes.com">  The New York Times Learning Network </a> , our email newsletters and other applications owned and operated by The New York Times Company (collectively referr

In [73]:
#Parse out the URL
url = 'http://med.net/this/that?thisthat/something.html'
'.'.join(url.split('//')[-1].split('/')[0].split('.')[-2:])
# >>> url.split("//")[-1].split("/")[0]
# 'stackoverflow.com'
# >>> url = "stackoverflow.com/questions/9626535/get-domain-name-from-url"
# >>> url.split("//")[-1].split("/")[0]
# 'stackoverflow.com'

'med.net'

## Define a new url segmenter to do all this.

In [101]:
import requests
from bs4 import BeautifulSoup

def url_input_parser(url):
    """
    Input is url as a string. Parses body text and returns
    string of all paragraph elements in doc.
    Secondarily, searches the input url for its domain and returns
    domain in the form 'google.com'.
    """
    
    # Request URL
    page = requests.get(url)
    
    # TODO
    # Check that we got a good response and throw an error if not
    
    content = BeautifulSoup(page.content, 'html.parser')
    
    # Parse URL paragraphs
    pars = [p.prettify() for p in content.find_all('p')]
    pars = '\n'.join(pars)

    # Parse domain
    domain = '.'.join(url.split('//')[-1].split('/')[0].split('.')[-2:])
    return pars, domain

In [102]:
testurl = 'http://www.facebook.com'
doc, domain = url_input_parser(testurl)
print(domain)
print(doc)

facebook.com
<p>
 Please enable JavaScript on your browser or upgrade to a JavaScript-capable browser to register for Facebook.
</p>
<p class="_58mv">
 By clicking Create Account, you agree to our
 <a href="/legal/terms" id="terms-link" rel="nofollow" target="_blank">
  Terms
 </a>
 and that you have read our
 <a href="/about/privacy" id="privacy-link" rel="nofollow" target="_blank">
  Data Policy
 </a>
 , including our
 <a href="/policies/cookies/" id="cookie-use-link" rel="nofollow" target="_blank">
  Cookie Use
 </a>
 . You may receive SMS Notifications from Facebook and can opt out at any time.
</p>


## Parsing the results

In [103]:
def text_paragraph_segmenter(doc):
    """
    input: doc as string
    output: list of paragraphs with blank lines removed
    """

    lines = doc.split('\n')
    return [line for line in lines if line.strip() != '']

In [116]:
import pandas as pd
segments = text_paragraph_segmenter(doc)
pd.DataFrame(segments).to_clipboard(index=False)

['<p>',
 ' Please enable JavaScript on your browser or upgrade to a JavaScript-capable browser to register for Facebook.',
 '</p>',
 '<p class="_58mv">',
 ' By clicking Create Account, you agree to our',
 ' <a href="/legal/terms" id="terms-link" rel="nofollow" target="_blank">',
 '  Terms',
 ' </a>',
 ' and that you have read our',
 ' <a href="/about/privacy" id="privacy-link" rel="nofollow" target="_blank">',
 '  Data Policy',
 ' </a>',
 ' , including our',
 ' <a href="/policies/cookies/" id="cookie-use-link" rel="nofollow" target="_blank">',
 '  Cookie Use',
 ' </a>',
 ' . You may receive SMS Notifications from Facebook and can opt out at any time.',
 '</p>']

In [119]:
test=segments[3]
re.sub(r'\<.*\>','',test)

''

## Pre-processing policy before the good stuff

In [124]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import string
def text_process_policy(doc):
    """
    Takes in doc as string and returns a processed string
    by performing the following steps:
    1. remove HTML tags
    2. remove punctuation
    3. remove stopwords
    4. stemming
    5. remove any blank lines
    """
    sn = SnowballStemmer(language='english')

    s = re.sub(r'\<.*\>','',doc)
    lst = [char for char in s if char not in string.punctuation]
    lst = ''.join(lst)
    lst = [word.lower() for word in lst.split() if word.lower() not in stopwords.words('english')]
    lst = [sn.stem(word) for word in lst]
    return ' '.join(lst)


In [125]:
segments_processed = [text_process_policy(segment) for segment in segments]
print(segments_processed)

['', 'pleas enabl javascript browser upgrad javascriptcap browser regist facebook', '', '', 'click creat account agre', '', 'term', '', 'read', '', 'data polici', '', 'includ', '', 'cooki use', '', 'may receiv sms notif facebook opt time', '']


In [133]:
no_blanks = [segment for segment in segments_processed if segment.strip() != '']
no_blanks

['pleas enabl javascript browser upgrad javascriptcap browser regist facebook',
 'click creat account agre',
 'term',
 'read',
 'data polici',
 'includ',
 'cooki use',
 'may receiv sms notif facebook opt time']

In [134]:
no_shorties = [segment for segment in no_blanks if len(segment) >= 20]
no_shorties

['pleas enabl javascript browser upgrad javascriptcap browser regist facebook',
 'click creat account agre',
 'may receiv sms notif facebook opt time']